# Backtesting a Trading Strategy Part 2

## Or How to Retrieve S&P Constituents Historical Data Using Python Package pandas-datareader

## Introduction
Backtesting is a tool we can use to measure the performance of a trading strategy using historical data. The backtesting process can be broken down in three parts: first, we need to decide the universe of securities where we invest in (e.g. equity or fixed income? US or emerging markets?); second, we need to retrieve historical data for the universe of securities; and last part is where we implement our trading strategy using the historical data collected. In the previous article, we completed the first step in the backtesting process of defining the universe of stocks, namely the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices. In this article, we proceed to the second step of the backtesting process, that is, we retrieve historical data for each constituent on every index. 

Below we load the tickers of S&P 500, S&P 400 and S&P 600 indices retrieved from the previous article. After, we use Python package [pandas-datareader](https://pandas-datareader.readthedocs.io) which allows us to collect historical market data from providers such as the [Investor Exchange (IEX)](https://iextrading.com), [Morningstar](https://www.morningstar.com), or [Robinhood](https://robinhood.com). We choose IEX as our data source since it provides 5 year historical data.

## Retrieve Historical Data

You can find the code below on https://github.com/DinodC/backtesting-mean-reverting-strategy.

Import packages

In [1]:
import pandas as pd

In [2]:
from pandas import Series, DataFrame

In [3]:
import pickle

In [4]:
import pandas_datareader.data as web

### S&P Constituents Tickers
In this section, we load the lists pickled from the last article.

Set an id for each index

In [5]:
id = ['sp500', 'sp400', 'sp600']

Create a dictionary to map each id to a tickers file

In [6]:
input_file = {'sp500': 'sp500_barchart.pickle',
              'sp400': 'sp400_barchart.pickle', 
              'sp600': 'sp600_barchart.pickle'} 

Define a dictionary to map each id to a tickers list

In [7]:
sp500_tickers = []
sp400_tickers = []
sp600_tickers = []
sp_tickers = {'sp500': sp500_tickers,
              'sp400': sp400_tickers,
              'sp600': sp600_tickers}

Fill the tickers lists

In [8]:
for i in input_file:
    with open(input_file[i], 'rb') as f:
        
        # Update tickers list        
        sp_tickers[i] = pickle.load(f)

        # Sort tickers list
        sp_tickers[i].sort()
        
    f.close()

### S&P Constituents Historical Data 

Define dictionary of historical data

In [9]:
sp500_data = pd.DataFrame()
sp400_data = pd.DataFrame()
sp600_data = pd.DataFrame()
sp_data = {'sp500': sp500_data,
           'sp400': sp400_data,
           'sp600': sp600_data}

Set the start and date of the historical data

In [10]:
start_date = '2014-01-01'
end_date = '2020-01-01'

Set the source [Investors Exchange (IEX)](https://iextrading.com) to be used

In [11]:
source = 'iex'

Create a dictionary to map each id to an output file

In [12]:
output_file = {'sp500': 'sp500_data.pickle',
               'sp400': 'sp400_data.pickle',
               'sp600': 'sp600_data.pickle'}

Retrieve historical data for each constituent of each S&P index

In [13]:
for i in output_file:
    
    # Retrieve historical data 
    # Note that we set number of tickers to < 100 because DataReader gives error when number of tickers > 100
    data1 = web.DataReader(sp_tickers[i][:98], source, start_date, end_date)
    data2 = web.DataReader(sp_tickers[i][98:198], source, start_date, end_date)
    data3 = web.DataReader(sp_tickers[i][198:298], source, start_date, end_date)
    data4 = web.DataReader(sp_tickers[i][298:398], source, start_date, end_date)
    data5 = web.DataReader(sp_tickers[i][398:498], source, start_date, end_date)
    if i == 'sp400':
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5], axis=1, sort=True)
    if i == 'sp500':
        data6 = web.DataReader(sp_tickers[i][498:], source, start_date, end_date)
        # Concatenate historical data        
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6], axis=1, sort=True)
    elif i == 'sp600':
        data6 = web.DataReader(sp_tickers[i][498:598], source, start_date, end_date)
        data7 = web.DataReader(sp_tickers[i][598:], source, start_date, end_date)
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6, data7], axis=1, sort=True)        
    else:
        pass
            
    # Convert index to datetime
    sp_data[i].index = pd.to_datetime(sp_data[i].index)
    
    # Save historical data to file
    with open(output_file[i], 'wb') as f:
        pickle.dump(sp_data[i], f)
    f.close()

## Constituents Close Prices

### S&P 500 Index

Eyeball

In [14]:
sp_data['sp500'].close.shape

(1258, 505)

In [15]:
sp_data['sp500'].close.head()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2014-05-09,37.7227,36.4477,123.5028,76.9614,43.4394,60.6682,21.59,35.0552,71.1754,59.59,...,25.9390,41.3795,85.0965,44.7737,26.7522,34.7281,49.9143,95.9139,27.4130,29.4467
2014-05-12,38.4174,37.5241,124.1869,77.9193,43.5308,61.6766,21.92,35.3070,71.5821,60.70,...,25.7028,42.1741,85.3302,45.0550,27.1364,35.5124,49.7835,96.7773,27.9511,29.7071
2014-05-13,38.7034,37.4479,122.3625,78.0415,43.3895,61.8431,21.55,35.6576,72.0251,60.81,...,25.7872,41.2278,85.4387,45.1908,27.3171,35.4471,50.0059,97.1610,27.5924,29.6299
2014-05-14,38.0360,37.0002,122.0948,78.0560,43.9464,62.2594,21.55,35.9004,71.2748,60.88,...,26.0234,40.9064,85.3803,44.7834,27.0686,35.2603,49.6070,96.8253,26.8278,29.5431
2014-05-15,37.1098,36.3810,123.1756,77.3922,43.7968,62.6632,21.20,35.2800,70.8861,60.20,...,25.7956,40.5850,84.1199,44.5893,26.7974,34.7654,48.9596,96.0770,26.6862,29.3696


In [16]:
sp_data['sp500'].close.tail()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2019-05-02,78.20,33.8523,166.78,209.15,78.47,77.06,264.77,78.78,179.39,279.64,...,55.92,118.90,77.29,51.50,32.95,79.54,101.74,123.21,49.48,103.15
2019-05-03,79.29,34.6899,163.27,211.75,78.71,79.14,271.75,78.69,176.98,285.58,...,56.58,119.02,77.47,55.05,32.95,82.29,102.72,124.31,50.05,103.75
2019-05-06,79.35,34.6500,161.99,208.48,79.26,78.31,268.95,79.07,176.26,283.66,...,56.51,118.81,77.13,54.73,32.65,80.08,102.41,125.55,49.68,103.33
2019-05-07,76.67,33.9100,160.66,202.86,77.95,77.46,261.98,76.91,173.94,277.07,...,56.58,117.81,76.72,54.83,32.67,79.17,101.47,123.39,48.71,101.37
2019-05-08,76.61,33.7500,158.62,202.90,77.99,78.72,260.27,76.22,173.82,276.77,...,55.88,117.69,76.84,54.95,32.09,79.13,100.49,122.56,48.19,101.86


In [17]:
sp_data['sp500'].close.describe()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,51.467024,41.343656,144.824652,133.794584,66.064139,84.739733,158.525886,48.392377,116.636740,138.544952,...,38.786547,57.890999,76.467798,53.016833,26.795565,50.742943,65.957073,111.383727,36.627797,58.504963
std,13.768604,6.216657,24.250751,38.325308,17.881638,9.845917,117.226499,12.213183,30.722033,68.878061,...,8.124638,21.478398,4.883311,7.892281,3.139932,16.191694,15.459884,10.036876,10.820941,19.725507
min,32.258600,24.539800,79.168700,76.961400,42.066600,60.668200,20.880000,33.935700,68.852300,59.590000,...,25.247700,32.609200,59.643400,34.178400,18.532600,28.968100,44.366900,89.236100,19.008100,29.196000
25%,39.221225,36.834425,130.553250,103.314650,52.521875,77.600350,75.840000,39.159175,90.555525,80.567500,...,31.174375,41.414825,73.431000,46.986675,24.215550,35.094975,52.731125,102.304000,26.912550,44.327550
50%,45.631150,40.848200,149.187400,118.695800,58.031100,84.243350,116.970000,42.809100,111.583650,105.580000,...,38.622750,51.057700,76.828250,53.996800,26.573400,47.844950,61.595700,112.414150,31.156350,50.342300
75%,65.193450,46.119900,161.834300,166.081025,82.894300,90.312650,236.567500,57.500400,147.791425,197.712500,...,45.139600,68.123175,79.625850,59.650550,29.473700,66.772550,79.238550,118.954900,46.685425,76.949925
max,81.940000,57.586600,199.159900,230.275400,116.445400,108.207500,449.750000,79.733700,182.670000,289.250000,...,57.360000,139.720000,87.124800,67.795300,35.000000,83.820000,104.390000,130.912800,57.511000,103.750000


### S&P MidCap 400 Index

Eyeball

In [18]:
sp_data['sp400'].close.shape

(1275, 400)

In [19]:
sp_data['sp400'].close.head()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2014-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-09,31.6652,31.9756,42.21,13.4225,32.07,NaN,9.8177,48.8031,52.6060,42.2125,...,22.0474,21.94,43.5655,15.3995,28.3556,23.8807,23.75,407.6382,54.22,74.03
2014-05-12,32.7774,31.9510,43.03,13.9825,32.60,NaN,9.9030,49.1525,52.5488,42.1031,...,22.1535,22.06,44.9818,16.2560,28.7706,24.7804,24.59,412.2492,56.60,74.23
2014-05-13,32.2164,31.6063,43.06,13.8000,32.02,NaN,10.0738,49.0277,52.5870,41.9076,...,22.0474,21.88,44.5868,16.4166,28.7152,24.8187,23.79,412.9473,55.53,73.41


In [20]:
sp_data['sp400'].close.tail()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2019-05-02,56.94,47.15,32.26,34.98,33.11,23.90,24.04,102.49,74.93,80.30,...,38.39,20.31,108.52,85.60,43.71,14.39,64.10,655.30,40.19,205.94
2019-05-03,58.75,47.31,33.42,35.42,33.62,24.89,24.12,103.51,74.82,82.05,...,38.09,22.96,112.34,85.80,44.63,16.88,64.45,662.82,40.98,206.46
2019-05-06,59.21,47.11,33.82,35.44,33.68,24.35,23.52,104.22,74.15,82.99,...,37.80,23.14,111.50,87.16,44.58,16.63,65.48,666.31,40.73,206.36
2019-05-07,58.28,46.25,32.60,34.40,33.11,21.97,22.71,102.42,73.11,82.20,...,37.88,22.54,109.12,84.93,43.92,16.41,62.51,652.75,39.91,200.82
2019-05-08,57.57,45.91,32.56,34.12,33.74,22.03,22.92,102.01,73.00,80.82,...,37.32,22.41,108.25,84.28,43.69,15.40,62.68,663.53,40.57,200.05


In [21]:
sp_data['sp400'].close.describe()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,633.000000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,34.951951,39.217452,48.998919,22.336920,32.397027,52.542665,15.623913,77.034952,55.537789,59.215583,...,29.498735,29.819408,62.681340,30.624876,36.782826,24.768914,53.553370,534.299863,40.549269,104.380668
std,9.474130,4.988952,13.759206,4.049079,3.178473,20.129762,4.052675,21.216151,9.369185,13.551177,...,4.838681,23.446983,15.011134,25.081843,7.050096,8.713336,25.852954,71.294603,13.384499,41.295902
min,20.118600,28.081400,24.750000,13.280000,23.150000,12.570000,8.872200,46.728500,40.309000,38.117400,...,20.741700,3.780000,38.963800,9.233700,25.207900,6.475600,19.560000,400.952700,15.230000,46.930000
25%,25.957675,34.702850,38.800000,19.390000,30.370000,40.435800,12.782225,58.763400,47.042025,45.533175,...,24.339550,12.060000,48.495775,15.573700,30.984350,19.100425,33.210000,471.868100,32.002500,74.200000
50%,34.380800,39.909300,46.660000,22.055000,32.425000,58.764300,14.581900,68.115800,53.312150,58.312100,...,29.706150,21.115000,61.817800,19.171300,34.616700,23.906400,45.895000,535.896500,39.565000,90.710000
75%,42.126325,43.673550,59.717500,24.070000,34.537500,66.641100,17.854850,97.221700,63.243550,73.313825,...,33.338075,44.240000,74.387375,34.439375,42.889700,32.300300,64.747500,598.808925,45.815000,123.212500
max,59.210000,48.853300,82.970000,35.520000,40.130000,84.046800,28.421700,114.972900,74.930000,83.830000,...,39.060000,103.090000,112.340000,99.250000,54.884400,45.690400,114.540000,666.310000,84.960000,235.440000


## S&P SmallCap 600 Index

Eyeball

In [22]:
sp_data['sp600'].close.shape

(1258, 601)

In [23]:
sp_data['sp600'].close.head()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2014-05-09,17.57,19.6925,30.3797,35.88,13.26,19.7560,62.85,24.2267,NaN,6.48,...,NaN,19.9735,77.30,9.2092,51.2889,24.7151,NaN,18.3379,23.3449,28.28
2014-05-12,17.44,20.4546,30.1511,36.36,14.06,20.1009,64.66,24.8687,NaN,6.76,...,NaN,20.0868,79.30,9.2678,52.4801,25.1019,NaN,18.5836,24.3523,28.93
2014-05-13,17.26,19.8847,30.1248,37.40,13.64,19.9764,63.44,24.5703,NaN,6.56,...,NaN,19.8683,79.31,9.1571,52.0263,24.7246,NaN,18.5243,23.6579,28.52
2014-05-14,17.22,19.2827,29.9577,36.51,13.01,19.4878,62.94,23.9554,NaN,6.24,...,11.9624,19.8279,77.66,9.1115,50.9485,24.1208,NaN,18.1260,22.7092,27.72
2014-05-15,18.15,19.1354,29.7819,36.49,13.05,19.0566,62.47,23.7474,NaN,6.24,...,12.4454,19.8764,77.35,9.0529,50.2185,24.0831,NaN,18.0921,22.4745,27.35


In [24]:
sp_data['sp600'].close.tail()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2019-05-02,12.09,49.52,46.12,45.91,64.26,36.75,79.07,37.42,30.29,21.27,...,4.39,27.89,129.49,12.50,84.36,37.04,22.31,24.88,16.14,25.79
2019-05-03,12.60,51.81,46.22,46.85,65.30,37.58,80.45,38.36,37.43,21.96,...,4.77,28.36,133.86,12.73,86.49,37.50,22.99,25.47,17.61,26.94
2019-05-06,12.69,49.35,46.18,44.95,65.84,37.39,80.58,38.41,37.62,22.08,...,4.86,28.36,132.62,12.78,87.51,36.98,23.00,25.08,17.02,26.53
2019-05-07,12.08,47.38,45.06,43.66,64.64,36.63,79.80,37.96,36.60,21.96,...,4.77,27.66,130.74,12.62,85.18,35.71,22.62,24.73,16.45,26.12
2019-05-08,12.06,47.35,45.31,43.13,67.00,36.25,79.84,37.93,36.29,19.08,...,4.73,27.59,131.16,12.52,84.56,35.25,22.68,24.56,16.31,25.95


In [25]:
sp_data['sp600'].close.describe()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,130.000000,1258.000000,...,1254.000000,1258.00000,1258.000000,1258.000000,1258.000000,1258.000000,1072.000000,1258.000000,1258.000000,1258.000000
mean,26.698299,29.224559,37.109340,49.388490,30.650918,36.185047,67.231924,32.523740,29.667508,15.859793,...,7.591375,26.06919,74.501331,10.776059,63.070520,26.637300,17.379708,26.422728,18.881719,22.917444
std,16.993067,7.540154,3.538491,10.202854,15.357067,10.727266,10.031661,5.652387,2.884428,6.630228,...,1.937716,3.23872,27.062866,1.389293,10.452208,5.747909,3.191832,6.741623,4.449880,6.830660
min,8.380000,16.343700,29.352300,31.400000,10.500000,18.903300,45.070000,22.347500,20.930200,6.240000,...,4.390000,19.78740,26.700000,7.392600,43.328400,14.741000,10.508700,12.051000,8.168700,11.450000
25%,15.060000,22.108125,35.180575,40.910000,21.952500,25.651450,59.167500,27.874650,28.616775,10.520000,...,6.081100,22.95650,50.137500,9.812600,54.135425,23.000950,14.881325,21.071525,16.473675,17.500000
50%,20.035000,29.230200,37.422800,49.150000,25.005000,34.638350,67.240000,31.102800,29.716800,13.650000,...,7.017050,26.39365,76.692500,10.822850,60.847400,26.401450,17.724600,26.908300,19.274100,22.075000
75%,33.927500,35.262375,39.002725,56.542500,33.810000,45.666850,73.522500,37.398775,31.059900,20.950000,...,8.848850,29.16485,97.620000,11.707150,71.798750,30.380675,19.508725,30.757175,22.262275,27.030000
max,99.610000,51.810000,46.770000,74.000000,74.890000,57.403000,95.540000,43.220900,37.620000,36.625000,...,12.678000,31.60290,133.860000,14.214700,87.510000,39.469500,24.438200,42.148900,30.787100,41.400000


## Summary

In this article, we loaded the constituents' symbols from the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices, and then passed them to pandas-datareader to retrieve historical data from the Investors Exchange. The data collection is a relatively straightforward operation, only requiring a couple of minutes to gather 5 year historical data of all the constituents. In the next article, we code a simple trading strategy and backtest it using the historical data just collected.